In [1]:
import wikipedia


In [19]:
search=input('enter the domain name  : ')

enter the domain name  : indian general elections


In [20]:
print('top 10 results for your \n\n')
wikipedia.search(search)

top 10 results for your 




['2019 Indian general election',
 '2014 Indian general election',
 'Next Indian general election',
 '1984 Indian general election',
 'Elections in India',
 '2009 Indian general election',
 '2004 Indian general election',
 '1951–52 Indian general election',
 '1991 Indian general election',
 '1998 Indian general election']

In [24]:
n = int(input('enter the number for which you want to see the summary : '))
text= wikipedia.page(wikipedia.search(search)[n-1]).summary
text

enter the number for which you want to see the summary : 3


'The next Indian general election is expected to be held in or before May 2024 to elect the members of 18th Lok Sabha.\n\n'

In [10]:
import spacy
from spacy import displacy

In [11]:
nlp = spacy.load('en_core_web_trf')

In [12]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud, STOPWORDS
from spacy.util import minibatch, compounding
import matplotlib.pyplot as plt
import re
import random
from pathlib import Path

In [13]:
df = pd.read_csv(r"C:\Users\satya\Downloads\drugsComTrain_raw.csv")
df

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37
...,...,...,...,...,...,...,...
161292,191035,Campral,Alcohol Dependence,"""I wrote my first report in Mid-October of 201...",10,31-May-15,125
161293,127085,Metoclopramide,Nausea/Vomiting,"""I was given this in IV before surgey. I immed...",1,1-Nov-11,34
161294,187382,Orencia,Rheumatoid Arthritis,"""Limited improvement after 4 months, developed...",2,15-Mar-14,35
161295,47128,Thyroid desiccated,Underactive Thyroid,"""I&#039;ve been on thyroid medication 49 years...",10,19-Sep-15,79


In [14]:
df.shape

(161297, 7)

In [15]:
example1 = 'James went to London to buy Orencia last year 2019'
doc1 = nlp(example1)
displacy.render(doc1,style='ent')

C:\Users\satya\anaconda3\lib\site-packages\torch\amp\autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [16]:
def process_review(review):
    processed_token = []
    for token in review.split():
        token = ''.join(e.lower() for e in token if e.isalnum())
        processed_token.append(token)
    return ' '.join(processed_token)

In [17]:
all_drugs = df['drugName'].unique().tolist()
all_drugs = [x.lower() for x in all_drugs]
all_drugs[:10]

['valsartan',
 'guanfacine',
 'lybrel',
 'ortho evra',
 'buprenorphine / naloxone',
 'cialis',
 'levonorgestrel',
 'aripiprazole',
 'keppra',
 'ethinyl estradiol / levonorgestrel']

In [18]:
count = 0
TRAIN_DATA = []
for _, item in df.iterrows():
    ent_dict = {}
    if count < 1000:
        review = process_review(item['review'])
        #Locate drugs and their positions once and add to the visited items.
        visited_items = []
        entities = []
        for token in review.split():
            if token in all_drugs:
                for i in re.finditer(token, review):
                    if token not in visited_items:
                        entity = (i.span()[0], i.span()[1], 'DRUG')
                        visited_items.append(token)
                        entities.append(entity)
        if len(entities) > 0:
            ent_dict['entities'] = entities
            train_item = (review, ent_dict)
            TRAIN_DATA.append(train_item)
            count+=1
len(TRAIN_DATA)

1000

In [19]:
#TRAIN_DATA = TRAIN_DATA[:3]

In [20]:
nlp.pipe_names

['transformer', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [21]:
ner = nlp.get_pipe('ner')
for _, annotations in TRAIN_DATA:
      for ent in annotations.get("entities"):
        ner.add_label(ent[2])

In [22]:
pipe_exceptions = ["ner", "transformer"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [ ]:
from spacy.training import Example
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(30):

    # shuufling examples  before every iteration
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=3)
    for batch in batches:
        texts, annotations = zip(*batch)
        #texts, annotations = zip(*batch)
            
        example = []
        # Update the model with iterating each text
        for i in range(len(texts)):
            doc = nlp.make_doc(texts[i])
            example.append(Example.from_dict(doc, annotations[i]))

            # Update the model
            nlp.update(example, drop=0.5,losses=losses)
        print("Losses", losses)
        #print(texts)